In [ ]:
import numpy as np
import pandas as pd
import datetime
import copy
from sklearn import tree
from keras.models import Sequential
from keras.layers import LSTM, Dense
import tensorflow as tf
from tensorflow import keras
from keras.layers import TimeDistributed, Conv1D, MaxPool1D, Flatten, LSTM, Dense, AveragePooling1D, Dropout, Conv2D, MaxPool2D
from keras.models import Sequential
from keras.regularizers import l2
import sklearn.metrics as metrics
from pyarrow import feather

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def data_processing(df, NUM_FEATURES):
   y_train = df['Label']
   flow_id = df['flow_id']

   df = df/255

   X_train = df.drop(['Label', 'flow_id'], axis=1)
   X_train = X_train.to_numpy()

   X_train = X_train.reshape(-1, 20, NUM_FEATURES)
   y_train = y_train.to_numpy()

   y_train = y_train.reshape(-1,20)[:,-1]
   return X_train, y_train

def DoiNhan(label: np.array):
  for i in range(len(label)):
    if label[i] == 14 or label[i] == 16 or label[i] == 13:
      label[i] = 2
    elif label[i] == 11 or label[i] == 5 or label[i] == 10:
      label[i] = 1
    elif label[i] == 4 or label[i] == 8 or label[i] == 9 or label[i] == 1 or label[i] == 7:
      label[i] = 0

In [ ]:
def MultiSourceTrAdaBoost(trans_S, Multi_trans_A, label_S, Multi_label_A, test, N=3, NUM_CLASSES=3):
    """
    Parameters
    ----------
    trans_S : feature matrix of same-distribution training data

    Multi_trans_A : dict, feature matrix of diff-distribution training data
    label_S : label of same-distribution training data, -1 or 1

    Multi_label_A : dict, label of diff-distribution training data, -1 or 1
    e.g.,
    Multi_label_A = {
    'label_A_1' :  label_1 ,
    'label_A_2' : label_2 ,
    ......
    }
    label_1 : label of diff-distribution training dataset 1, -1 or 1
    label_1 : label of diff-distribution training dataset 2, -1 or 1

    test : feature matrix of test data

    N : int, default=3
    the number of weak estimators

    """
    # prepare trans_A
    trans_A = list(Multi_trans_A.values())[0]

    if len(Multi_trans_A) == 1:
        pass
    else:
        for i in range(len(Multi_trans_A)-1):
            p = i + 1
            trans_A = np.concatenate((trans_A, list(Multi_trans_A.values())[p]), axis=0)
    # prepare label_A
    label_A = list(Multi_label_A.values())[0]
    if len(Multi_label_A) == 1:
        pass
    else:
        for i in range(len(Multi_label_A)-1):
            p = i + 1
            label_A = np.concatenate((label_A, list(Multi_label_A.values())[p]), axis=0)

    trans_data = np.concatenate((trans_A, trans_S), axis=0)
    trans_label = np.concatenate((label_A, label_S), axis=0)

    row_A = trans_A.shape[0]
    row_S = trans_S.shape[0]
    row_T = test.shape[0]

    if N >= row_A:
        print('The maximum of iterations should be smaller than ', row_A)

    test_data = np.concatenate((trans_data, test), axis=0)

    # Initialize the weights
    weights_A = np.ones([row_A, 1]) / row_A
    weights_S = np.ones([row_S, 1]) / row_S
    # one-dim column in the shape of ((row_A+row_S),1), column vector
    weights = np.concatenate((weights_A, weights_S), axis=0)

    alpha_S = 1/(1+np.sqrt(2*np.log(row_S + row_T)/N))


    # Save prediction labels and bata_t
    alpha_T = np.zeros([1, N])
    result_label = np.ones([row_A + row_S + row_T, N])
    # output label
    predict = np.zeros([row_T])
    print ('params initial finished.')
    print('='*60)

    trans_data = np.asarray(trans_data, order='C')
    trans_label = np.asarray(trans_label, order='C')
    test_data = np.asarray(test_data, order='C')

    for i in range(N):
        weights = calculate_ratio_weight(weights)

        result_label[:, i], error_rate , Source_index, start = Multi_train_classifier(Multi_trans_A, label_S,trans_data, trans_label, test_data, weights,row_A,row_S)
        # Avoiding overfitting
        if error_rate >= 0.667:
            error_rate = 0.6;

        if error_rate <= 1e-10:
            N = i
            break

        alpha_T[0, i] = np.log((1 - error_rate) / error_rate) + np.log(NUM_CLASSES - 1)
        C_T = NUM_CLASSES * (1 - error_rate)
        print('Iter {}-th result :'.format(i))
        print('The {}-th diff-distribution training dataset is chosen to transfer'.format(Source_index))
        print('error rate :', error_rate, '|| alpha_T :', np.log((1 - error_rate) / error_rate) + np.log(NUM_CLASSES - 1) )
        print('-'*60)

        # Changing the data weights of same-distribution training data
        for j in range(row_S):
          loss_t = np.sum(result_label[row_A + j, i] != label_S[j])
          weights[row_A + j] = weights[row_A + j] * np.exp(alpha_T[0, i] * loss_t)
        # Changing the data weights of diff-distribution training data
        for j in range( len(list(Multi_trans_A.values())[Source_index]) ):
            loc = start + j
            loss_ = np.sum(result_label[loc, i] != label_A[loc])
            weights[loc] = C_T * weights[loc] * np.exp(alpha_S * loss_)

    for i in range(row_T):
      predict[i] = result_label[row_A + row_S + i, :][np.argmax(alpha_T[0, :])]

    print(predict)
    print(predict.shape)

    print("MultiSourceTrAdaBoost is done")
    print('='*60)
    print('The prediction labels of test data are :')
    print(predict)
    return predict


def calculate_ratio_weight(weights):
    total = np.sum(weights)
    return np.asarray(weights / total, order='C')


# def train_classifier(trans_data, trans_label, test_data, ratio_weight):
#     clf = tree.DecisionTreeClassifier(criterion="gini", max_depth = 2, max_features="log2", splitter="best",random_state=0)
#     clf.fit(trans_data, trans_label, sample_weight=ratio_weight[:, 0])
#     return clf.predict(test_data)

# def train_LSTM(trans_data, trans_label, test_data, ratio_weight):

#     # Define model architecture
#     model = Sequential()
#     model.add(LSTM(64, input_shape=(trans_data.shape[1], 1),return_sequences=True))
#     # model.add(Dropout(0.2))
#     model.add(LSTM(32,activation='relu'))
#     model.add(Dense(16, activation="relu"))
#     model.add(Dense(NUM_CLASSES, activation='softmax'))

#     # model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#     # history = model.fit(trans_data, trans_label, epochs=NUM_EPOCHS, sample_weight=ratio_weight[:, 0])

#     model.compile(optimizer=tf.keras.optimizers.Adam(
#     learning_rate=client_lr), loss=tf.keras.losses.SparseCategoricalCrossentropy(),
#     metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
#     history = model.fit(trans_data, trans_label, epochs=NUM_EPOCHS, sample_weight=ratio_weight[:, 0], )

#     # y_pred = model.predict(test_data)

#     predictions = model.predict(test_data)
#     flow_pred = np.argmax(predictions, axis=-1)
#     return flow_pred

def train_classifier(trans_data, trans_label, test_data, ratio_weight):
    NUM_EPOCHS = 2
    client_lr = 3e-4
    BATCH_SIZE = 32

    model = Sequential()

    model.add(Conv2D(filters=128, kernel_size=(5, 5), padding='Same',
                     activation='relu', input_shape=(20,128,1)))
    model.add(Conv2D(filters=64 , kernel_size=(5, 5), padding='Same',
                     activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='Same',
                     activation='relu'))
    model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='Same',
                     activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='Same',
                     activation='relu'))
    model.add(Conv2D(filters=16, kernel_size=(3, 3), padding='Same',
                     activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))


    model.add(Flatten())
    model.add(Dense(256, activation="relu"))
    model.add(Dropout(0.1))
    model.add(Dense(3, activation="softmax"))

    model.compile(optimizer=tf.keras.optimizers.Adam(
    learning_rate=client_lr), loss=tf.keras.losses.SparseCategoricalCrossentropy(),

    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
    model.fit(trans_data, trans_label, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE,
                    shuffle=True, sample_weight=ratio_weight[:, 0])

    predictions = model.predict(test_data, verbose=2, batch_size=BATCH_SIZE)

    flow_pred = np.argmax(predictions, axis=-1)

    return flow_pred

def Multi_train_classifier(Multi_trans_A,label_S, trans_data, trans_label, test_data, weights,row_A,row_S):
    _result_label = np.ones([len(test_data), len(Multi_trans_A)])
    error_record = []
    start_record = []
    start = 0
    for item in range(len(Multi_trans_A)):
        start_record.append(start)
        sub_dataset = list(Multi_trans_A.values())[item]
        data_dim = len(sub_dataset)
        # train a classifier with the 'item'-th data source
        _trans_data = np.concatenate((trans_data[start : start + data_dim], trans_data[row_A:row_A + row_S]), axis=0)
        _trans_label = np.concatenate((trans_label[start : start + data_dim], trans_label[row_A:row_A + row_S]), axis=0)
        _ratio_weight = np.concatenate((weights[start : start + data_dim], weights[row_A:row_A + row_S]), axis=0)
        _result_label[:, item] = train_classifier(_trans_data, _trans_label, test_data, _ratio_weight)
        start += data_dim
        # cal error rate
        _error_rate = calculate_error_rate(label_S, _result_label[row_A:row_A + row_S, item],weights[row_A:row_A + row_S, :])

        error_record.append(_error_rate)
    error_record = np.array(error_record)
    # choise the best classifier
    classifier_index = np.random.choice(np.flatnonzero(error_record == error_record.min()))
    return _result_label[:,classifier_index], error_record[classifier_index], classifier_index,start_record[classifier_index]

def calculate_error_rate(label_R, label_P, weight):
    total = np.sum(weight)
    loss = np.sum(label_P != label_R)
    return np.sum(weight[:, 0] / total * loss)

In [ ]:
def train_and_test():
  path = '/content/drive/MyDrive/Fixed_data/'
  test_name = 'd1test'
  train_name ='d1train'

  diff = feather.read_feather('/content/drive/MyDrive/Fixed_data/domain2.feather')
  diff_distribution, diff_distribution_label = data_processing(diff, 128)
  DoiNhan(diff_distribution_label)

  Multi_trans_A = {
    'trans_A_1' : diff_distribution

    }
  Multi_label_A = {
    'label_A_1' :  diff_distribution_label
    }

  print('source data initial finished')
  for i in range(1, 5):
    print('Start training with ' + str(i) + '% train data and ' + str(100 - i) + '% test data')
    same = feather.read_feather(path + train_name + str(i))
    df_tg = feather.read_feather(path + test_name + str(i))

    same_distribution, same_distribution_label = data_processing(same, 128)
    tg, tg_label = data_processing(df_tg, 128)

    DoiNhan(same_distribution_label)
    DoiNhan(tg_label)

    trans_S = same_distribution
    label_S = same_distribution_label
    test = tg
    label_T = tg_label

    predict = MultiSourceTrAdaBoost(trans_S, Multi_trans_A, label_S, Multi_label_A, test)
    print(metrics.classification_report(predict, label_T))

In [ ]:
train_and_test()

source data initial finished
Start training with 1% train data and 99% test data
params initial finished.
Epoch 1/2
198/198 [==============================] - 612s 3s/step - loss: 1.6456e-04 - sparse_categorical_accuracy: 0.4998
Epoch 2/2
198/198 [==============================] - 605s 3s/step - loss: 1.2456e-04 - sparse_categorical_accuracy: 0.6958
474/474 - 381s - 381s/epoch - 805ms/step
Iter 0-th result :
The 0-th diff-distribution training dataset is chosen to transfer
error rate : 0.6 || alpha_T : 0.287682072451781
------------------------------------------------------------
Epoch 1/2
 36/198 [====>.........................] - ETA: 8:03 - loss: 1.4399e-04 - sparse_categorical_accuracy: 0.3047

In [ ]:
while True:
  pass